<a href="https://colab.research.google.com/github/AbdhMohammady/DataScience/blob/main/Information_gain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font size=20>Information Gain</font></b>

In this notebook, we are going to implement the method to calculate information gain based on entropy and Gini method.
This is an educational notebook and I request the readers to report any bugs to <b>AbdhMohammady@gmail.com</b> or leave comment on my github <b>https://github.com/AbdhMohammady</b>.
I tried to write this notebook so that it can be used for other datasets, currently this notebook can be used for datasets with categorical columns and two-class for target column.
At the end of the notebook, after obtaining the highest information gain,you can use this column as the root of the decision tree algorithm.

The entropy of the original dataset is calculated based on the target column named 'Buy' and this column has two class 'Yes' and 'No'.

Where :

 D is our dataset 
 
 number of rows in the dataset is n(dataset)
 
 frequency of class ‘i’ in the target column is n(class i)
 
 then probability of class i in the dataset is :
$$p(class i) = {p}_{i}=\frac{{n}({class} {i})}{{n}({dataset})}$$

and entropy of dataset is :
$${Entropy}\left({D}\right)=\ -\sum_{{i}=\mathbf{1}}^{{m}}{{p}_{i}{Ln}({p}_{i})}$$


Suppose A is a column of the dataset and Aj, j:1... k are the categories in this column.
for each Aj, suppose pi is probability of class i in the Aj then:

$${Entropy}\left({A}_{j}\right)=\ -\sum_{{j}=\mathbf{1}}^{{k}}{{p}_{i}({A}_{j}){Ln}({p}_{i}({A}_{j}))}$$

and if p(Aj) is probability of Aj in the A, then entropy of A is :

$${Entropy}\left({A}\right)=\ \sum_{{j}=\mathbf{1}}^{{m}}{{p}({A}_{j}){Entropy}({A}_{j})}$$

And as final result, information gain of A is:

$${Gain}({A})\ =\ {Entropy}({D})-\ {Entropy}({A})$$

This calculation is repeated for all columns, and at the end, the column with the highest value of information gain is selected as the root of the decision tree algoritm. This process is implemented in the below.

*   When the number of classes in a variable is large, increasing the number of sentences in summation causes the value of the information gain of this variable to increase and leads to an illogical result. To deal with this problem, split information is used.

Where p(Ai) is probabilty of Ai in the dataset : 
$${SplitInformation}({A})\ =\ -\ \sum_{{i}=\mathbf{1}}^{{k}}{{p}({A}_{i}){Ln}({p}({A}_{i}))}$$

and 'Gain Ratio' is used instead of 'Information Gain' :

$${GainRatio}({A})=\frac{{Gain}({A})}{{SplitInformation}({A})}$$


contact me:

https://github.com/AbdhMohammady

abdhmohammady@gmail.com

In [ ]:
import pandas # to working with dataset
import math   # to working with math functions like log10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#This is a very sample dataset to learn Gain information rules
df = pandas.read_csv('/content/drive/MyDrive/DATA/Information-gain-sample.csv')

# number of record we read by pandas from our dataset, 
# we use this value as size of the sample to claculte probability of each class
n_records = len(df)

# display all data
df.head()


,Credit,Age,Income,Education,Buy
0,Good,Young,High,Good,No
1,Excellent,Young,High,Good,No
2,Good,Middle,High,Good,Yes
3,Good,Old,Middle,Good,Yes
4,Good,Old,High,Bad,Yes


In [ ]:
group_object = df.groupby(by="Buy")

buy_groups_dict = group_object.groups

print("Buy groups dictionary:\n",buy_groups_dict)

group_object.describe()


Buy groups dictionary:
 {'No': [0, 1, 5, 7, 13], 'Yes': [2, 3, 4, 6, 8, 9, 10, 11, 12]}


Credit                          Age                     Income         \
     count unique        top freq count unique     top freq  count unique   
Buy                                                                         
No       5      2  Excellent    3     5      2   Young    3      5      2   
Yes      9      2       Good    6     9      3  Middle    4      9      2   

               Education                    
      top freq     count unique   top freq  
Buy                                         
No   High    3         5      2  Good    4  
Yes  High    5         9      2   Bad    6

In [ ]:
#Compute entropy of the dataset

#Calculate the frequency of each item in the group

dataset_entropy = 0
dataset_gini = 0

target_probs = dict()

for key in buy_groups_dict:
    #list of real index of items in the dataset
    index_list = buy_groups_dict.get(str(key)).to_list()
    
    n_class_item = len(index_list)
    # Computes probability of class i in the target column.
    # this probability is p(class i)
    prob_class_item = n_class_item/n_records
    # It calculates the exponential of the probability of each class to obtain the Gini value of the dataset.
    dataset_gini += prob_class_item**2
    
    # calculate summation parts of entropy for dataset
    # this is store pi*Ln(pi) parts
    dataset_entropy += prob_class_item*math.log(prob_class_item)

#Entropy of the dataset - Entropy(D)
dataset_entropy = - dataset_entropy
# Gini of the dataset
dataset_gini = 1- dataset_gini

print("Dataset Entropy : ",dataset_entropy)

print("Dataset Gini    : ",dataset_gini)
    

Dataset Entropy :  0.6517565611726531
Dataset Gini    :  0.4591836734693877


Computing entropy of a column

In [ ]:
# get list of column name
columns = list(df.columns.values)

# no need target column to calculate entropy and gain information
del columns[4] 

columns

['Credit', 'Age', 'Income', 'Education']

In [ ]:

# initalize empty dictionary to store 'Gain Information' of each column 
info_dict = dict()

column_entropy = 0

for column_ in columns:
    
    column_entropy = 0
    column_gini    = 0
    #Groupping the column by categories
    group_object = df.groupby(by=column_)

    #the categories of the column
    categories = list(group_object.groups.keys())

    for category in categories:

        n_yes = 0
        n_no  = 0
        
        # Enumerate frequency of 'yes' and 'no' in the category
        for i in range(n_records):
            if df[column_][i]==category:
                if df['Buy'][i]=='Yes': n_yes+=1
                if df['Buy'][i]=='No' : n_no+=1

        #Number of subclass items
        n_category = n_yes + n_no

        p_yes = 0
        # Calculates probability of 'Yes' for this category
        if n_yes != 0 and n_category != 0 :p_yes = n_yes/n_category
        
        p_no  = 0
        # Calculates probability of 'No' for this category
        if n_no != 0 and n_category != 0 :p_no = n_no/n_category
        
        # Calculates Entropy(Aj)
        category_entropy = 0
        
        if p_yes != 0 : category_entropy = p_yes*math.log(p_yes)
        
        if p_no != 0 : category_entropy += p_no*math.log(p_no)
        
        #Calculates Gini(Aj)
        category_gini = 1 - (p_yes**2 + p_no**2)
        
        category_entropy = - category_entropy

        # Calculate p(Aj)Entropy(Aj) for each i and adds to summation
        column_entropy += (n_category/n_records)* category_entropy
        
        # Calculate p(Aj)Gini(Aj) for each i and adds to summation
        column_gini += (n_category/n_records)*category_gini
        
    # Information gain and gain ratio of current column
    # firts element of the value list is gain info
    # secund element of the value list is gain ratio
    # Third element of the value list is gini info
    gain__info = dataset_entropy - column_entropy
    gain_ratio = gain__info/column_entropy
    gini_info  = dataset_gini - column_gini
    
    info_dict[column_] = [gain__info,gain_ratio,gini_info]
    
    print("\n(InformationGain,GainRatio,GiniGain) = (",gain__info,",",gain_ratio,",",gini_info,")")
    


(InformationGain,GainRatio,GiniGain) = ( 0.033359115436214726 , 0.053944458642593444 , 0.030612244897959162 )

(InformationGain,GainRatio,GiniGain) = ( 0.17103394188032706 , 0.35578509314187645 , 0.11632653061224485 )

(InformationGain,GainRatio,GiniGain) = ( 0.0009286386703150074 , 0.0014268574506522823 , 0.000850340136054395 )

(InformationGain,GainRatio,GiniGain) = ( 0.10524434967821283 , 0.19257456185731267 , 0.09183673469387743 )


In [ ]:
#find item with maximom value in the dictionary using Gain Information
max_item = list(info_dict.keys())[0]

for item in info_dict:
    if info_dict[item][0]> info_dict[max_item][0]:
        max_item = item

print("The column with the highest information gain:\n\n\tColumn : '"
      + max_item+"'\n\tStandard method :",info_dict[max_item][0],
     "\n\tGain Ratio method:",info_dict[max_item][1],
     "\n\tGini method:",info_dict[max_item][2])

The column with the highest information gain:

	Column : 'Age'
	Standard method : 0.17103394188032706 
	Gain Ratio method: 0.35578509314187645 
	Gini method: 0.11632653061224485


<b>Now the column found can be used for the root of the decision tree algorithm</b>